In [ ]:
!pip install folium

In [ ]:
pip install pyttsx3

In [ ]:
pip install SpeechRecognition


In [ ]:
pip install pyaudio


In [ ]:
!pip install geopy

In [ ]:
import json

# Open the JSON file with the correct encoding
with open('csvjson (2).json', encoding='utf-8') as file:
    data = json.load(file)

for i, test in enumerate(data[:10000]):
    print(f"Item {i+1}: {test}")


In [ ]:
import csv
import pyttsx3
import speech_recognition as sr

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Set properties, if needed (rate, volume, voice)
engine.setProperty('rate', 150)  # Speed of speech
engine.setProperty('volume', 1)   # Volume (0.0 to 1.0)

# Convert text to speech
engine.say("請輸入目的地")

# Wait for the speech to finish
engine.runAndWait()

# Load data from CSV file
with open('kyle.csv', encoding='UTF-8') as csvfile:
    data = csv.reader(csvfile)
    dic = {}
    dic2 = {}
    for row in data:
        lst = [s for s in row[2].strip().split('區')]
        if lst[0] not in dic:
            dic[lst[0]] = 1
        else:
            dic[lst[0]] += 1
        if len(lst) > 1:
            road = [s for s in lst[1].split('路')]
            if (lst[0], road[0] + '路') not in dic2:
                dic2[(lst[0], road[0] + '路')] = 1
            else:
                dic2[(lst[0], road[0] + '路')] += 1

# Process data to determine dangerous areas
dic3 = {}
for location, data in dic2.items():
    if data >= 200:
        if location[0] not in dic3:
            dic3[location[0]] = location[1]
        else:
            dic3[location[0]] += (location[1])

# Initialize the recognizer
recognizer = sr.Recognizer()

# Use the default microphone as the audio source
with sr.Microphone() as source:
    print("請開始說話...")
    audio = recognizer.listen(source)

try:
    # Recognize speech using Google Speech Recognition
    place = recognizer.recognize_google(audio, language="zh-TW")
    print("您說的目的地是:", place)
except sr.UnknownValueError:
    print("抱歉，無法理解您說的話")
    place = ""
except sr.RequestError:
    print("抱歉，無法連接到語音識別服務")
    place = ""

# Process the recognized place
if place:
    lstt = [s for s in place.strip().split('區')]
    area = lstt[0]
    lstt2 = [s for s in lstt[1].strip().split('路')]
    street = lstt2[0]
    
    if dic[area] >= 2000:
        if dic3.get(area):
            lst_i = [s for s in dic3[area].split('路')]
            txt = f'此地巨危險，{area}區去年發生了{dic[area]}件車禍，並且位於'
            for i in lst_i:
                if len(i) != 0:
                    txt += f'{i}路'
            txt += '上的車禍件數十分多，用路人經過以上路段時要特別注意，詳細的車禍資訊可察看本站製作的各區交通分析'
        else:
            txt = f'此地巨危險，{area}區去年發生了{dic[area]}件車禍，用路人要特別注意，詳細的車禍資訊可察看本站製作的各區交通分析'
    else:
        if dic3.get(area):
            lst_i = [s for s in dic3[area].split('路')]
            txt = f'{area}區去年發生了{dic[area]}件車禍，但相較其他區域，此區相較安全，但經過'
            for i in lst_i:
                if len(i) != 0:
                    txt += f'{i}路'
            txt += '時要特別注意，詳細的車禍資訊可察看本站製作的各區交通分析'
        else:
            txt = f'{area}區去年發生了{dic[area]}件車禍，但相較其他區域，此區相較安全，詳細的車禍資訊可察看本站製作的各區交通分析'

    # Convert text to speech
    engine.say(txt)

    # Wait for the speech to finish
    engine.runAndWait()


In [ ]:
import folium
import csv
from folium import plugins
from ipywidgets import Dropdown, Output

point_map = folium.Map(location=[25.09108, 121.5598], zoom_start=15, width='100%', height='100%')
circle_map = folium.Map(location=[25.09108, 121.5598], zoom_start=15, width='100%', height='100%')
incidents = plugins.MarkerCluster().add_to(point_map)


dropdown = Dropdown(options=['point_map', 'circle_map'], description='Select Map:')
output = Output()

with open('kyle.csv', encoding='UTF-8') as csvfile:
    data = csv.reader(csvfile)
    for row in data:
        lst = [s for s in row[0].split('/')]
        if len(lst)>2:
            num = int(lst[1])
            if num == 12:
                if row[4] and row[3]:
                    latitude = float(row[4])
                    longitude = float(row[3])
                    point_map.add_child(folium.Marker(location=[latitude, longitude], popup=row[2]))
                    folium.Marker(location=[latitude, longitude],popup=row[2], icon=None).add_to(incidents)

        
def update_map(change):
    output.clear_output()
    with output:
        if change['new'] == 'point_map':
            display(point_map)
        elif change['new'] == 'circle_map':
            display(circle_map)

circle_map.add_child(incidents)
dropdown.observe(update_map, 'value')
display(dropdown)
display(output)